In [423]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

In [424]:
swm = pd.read_csv('swm_infrastructure.csv')
ses = pd.read_csv('ses_predictors_v2.csv')
lu = pd.read_csv('landuse_predictors_v4.csv')
df311 = pd.read_csv('311_cleaned.csv')

## Data exploration

In [425]:
swm.head()

,Unnamed: 0,cd,boro,date,year,month,REFUSETONSCOLLECTED,PAPERTONSCOLLECTED,MGPTONSCOLLECTED,RESORGANICSTONS,...,shape_area,wfr_avg_recycling,wfr_avg_refuse,wfr_avg_bulk,bigbelly,seasonal_wire_litter_basket,highend_basket,publicspacereceptacle,wire_basket,total_litter_baskets
0,0,BK01,BK,2016-11-01,2016,11,5250.4,527.8,402.8,16.9,...,0.001307,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0
1,1,BK01,BK,2016-12-01,2016,12,5409.5,726.3,501.0,11.7,...,0.001307,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0
2,2,BK01,BK,2017-01-01,2017,1,5189.9,589.1,455.2,14.9,...,0.001307,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0
3,3,BK01,BK,2017-02-01,2017,2,4705.4,505.6,395.8,10.5,...,0.001307,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0
4,4,BK01,BK,2017-03-01,2017,3,5476.7,563.0,453.3,10.6,...,0.001307,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0


In [426]:
swm.shape #checking that lengths of all the dfs align

(3540, 28)

In [427]:
ses.head()

,Unnamed: 0,Unnamed: 0.1,borocd,pct_bach_deg,over65_rate,under18_rate,pct_white_nh,poverty_rate,pct_hh_rent_burd,mean_commute,unemployment,crime_per_1000,pop_density,month,borough_no,cd,borough_code,community_district
0,0,0,201,11.3,9.0,29.7,2.1,32.3,49.4,42.5,6.4,21.7,42230.049029,2016-11-01,2,1,BX,BX01
1,1,1,202,14.2,9.2,27.5,1.9,32.3,53.8,44.9,6.4,12.3,23547.039187,2016-11-01,2,2,BX,BX02
2,2,2,203,14.1,9.2,29.1,1.2,31.4,48.0,43.9,6.6,13.6,49616.354945,2016-11-01,2,3,BX,BX03
3,3,3,206,12.6,9.6,28.3,6.1,31.4,55.1,44.4,8.4,12.0,54409.272313,2016-11-01,2,6,BX,BX06
4,4,4,204,15.8,9.7,27.7,1.9,34.7,54.0,41.9,6.4,14.6,73513.072409,2016-11-01,2,4,BX,BX04


In [428]:
ses.shape  #checking that lengths of all the dfs align

(3540, 18)

In [429]:
lu.head()

,month,community_district,numbldgs,unitstotal,asland_sqft,pct_clean_strts,pct_lot_area_open_space,pct_lot_area_public_facility_institution,pct_lot_area_res_1_2_family_bldg,pct_lot_area_res_multifamily_walkup,restaurant,business
0,11/1/16,MN01,1760,55988,79.282161,100.0,4.35,31.68,0.17,0.63,3295,683
1,11/1/16,MN02,5375,72165,100.324753,98.7,3.55,9.63,4.47,10.26,1271,1013
2,11/1/16,MN03,4708,94059,58.236802,93.4,14.95,11.15,0.32,8.66,580,947
3,11/1/16,MN04,3923,100090,105.402387,100.0,2.81,11.75,0.88,7.68,964,1014
4,11/1/16,MN05,3190,95425,516.066825,100.0,1.59,6.49,0.09,0.67,270,2083


In [430]:
lu.shape

(3540, 12)

In [431]:
df311.head()

,Unnamed: 0,CD,Year_Month,Year,Month,Count
0,0,BK01,2016-11,2016,11,50
1,1,BK01,2016-12,2016,12,36
2,2,BK01,2017-01,2017,1,51
3,3,BK01,2017-02,2017,2,61
4,4,BK01,2017-03,2017,3,67


In [432]:
df311.shape

(3523, 6)

## Data cleaning

### swm

In [433]:
swm.columns

Index(['Unnamed: 0', 'cd', 'boro', 'date', 'year', 'month',
       'REFUSETONSCOLLECTED', 'PAPERTONSCOLLECTED', 'MGPTONSCOLLECTED',
       'RESORGANICSTONS', 'fs_openhours', 'fs_open', 'trs_permits',
       'paper_baskets', 'mgp_baskets', 'total_number_of_households',
       'hh_compost_num', 'hh_compost_percent', 'shape_area',
       'wfr_avg_recycling', 'wfr_avg_refuse', 'wfr_avg_bulk', 'bigbelly',
       'seasonal_wire_litter_basket', 'highend_basket',
       'publicspacereceptacle', 'wire_basket', 'total_litter_baskets'],
      dtype='object')

In [434]:
swm['Year_Month'] = pd.to_datetime(swm['date']).dt.to_period('M')
swm = swm.drop(['Unnamed: 0','boro','date'],axis=1)

In [435]:
swm['Year_Month'].unique()

<PeriodArray>
['2016-11', '2016-12', '2017-01', '2017-02', '2017-03', '2017-04', '2017-05',
 '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12',
 '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07',
 '2018-08', '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02',
 '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09',
 '2019-10', '2019-11', '2019-12', '2020-01', '2020-02', '2020-03', '2020-04',
 '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11',
 '2020-12', '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
 '2021-07', '2021-08', '2021-09', '2021-10']
Length: 60, dtype: period[M]

In [436]:
swm.dtypes

cd                                object
year                               int64
month                              int64
REFUSETONSCOLLECTED              float64
PAPERTONSCOLLECTED               float64
MGPTONSCOLLECTED                 float64
RESORGANICSTONS                  float64
fs_openhours                     float64
fs_open                          float64
trs_permits                      float64
paper_baskets                    float64
mgp_baskets                      float64
total_number_of_households         int64
hh_compost_num                     int64
hh_compost_percent               float64
shape_area                       float64
wfr_avg_recycling                float64
wfr_avg_refuse                   float64
wfr_avg_bulk                     float64
bigbelly                         float64
seasonal_wire_litter_basket      float64
highend_basket                   float64
publicspacereceptacle            float64
wire_basket                      float64
total_litter_bas

In [437]:
swm.rename({'cd':'CD','year':'Year','month':'Month'},inplace=True,axis=1)
swm.head()

,CD,Year,Month,REFUSETONSCOLLECTED,PAPERTONSCOLLECTED,MGPTONSCOLLECTED,RESORGANICSTONS,fs_openhours,fs_open,trs_permits,...,wfr_avg_recycling,wfr_avg_refuse,wfr_avg_bulk,bigbelly,seasonal_wire_litter_basket,highend_basket,publicspacereceptacle,wire_basket,total_litter_baskets,Year_Month
0,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0,2016-11
1,BK01,2016,12,5409.5,726.3,501.0,11.7,333.25,6.0,14.0,...,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0,2016-12
2,BK01,2017,1,5189.9,589.1,455.2,14.9,333.25,6.0,14.0,...,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0,2017-01
3,BK01,2017,2,4705.4,505.6,395.8,10.5,333.25,6.0,14.0,...,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0,2017-02
4,BK01,2017,3,5476.7,563.0,453.3,10.6,333.25,6.0,14.0,...,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0,2017-03


### ses

In [438]:
ses['Year_Month'] = pd.to_datetime(ses['month']).dt.to_period('M')
ses['Year'] = ses['Year_Month'].dt.year
ses['Month'] = ses['Year_Month'].dt.month

In [439]:
ses = ses.drop(['Unnamed: 0','Unnamed: 0.1','borocd','borough_no','cd','borough_code'],axis=1)

In [440]:
ses.rename({'community_district':'CD'},inplace=True,axis=1)
ses.head()

,pct_bach_deg,over65_rate,under18_rate,pct_white_nh,poverty_rate,pct_hh_rent_burd,mean_commute,unemployment,crime_per_1000,pop_density,month,CD,Year_Month,Year,Month
0,11.3,9.0,29.7,2.1,32.3,49.4,42.5,6.4,21.7,42230.049029,2016-11-01,BX01,2016-11,2016,11
1,14.2,9.2,27.5,1.9,32.3,53.8,44.9,6.4,12.3,23547.039187,2016-11-01,BX02,2016-11,2016,11
2,14.1,9.2,29.1,1.2,31.4,48.0,43.9,6.6,13.6,49616.354945,2016-11-01,BX03,2016-11,2016,11
3,12.6,9.6,28.3,6.1,31.4,55.1,44.4,8.4,12.0,54409.272313,2016-11-01,BX06,2016-11,2016,11
4,15.8,9.7,27.7,1.9,34.7,54.0,41.9,6.4,14.6,73513.072409,2016-11-01,BX04,2016-11,2016,11


### lu

In [441]:
# extracting date & month 
lu['month'] = pd.to_datetime(lu['month'])
lu['Year'] = lu['month'].dt.year
lu['Month'] = lu['month'].dt.month 

In [442]:
lu['Year_Month'] = pd.to_datetime(ses['month']).dt.to_period('M')

In [443]:
lu.rename({'community_district':'CD'},inplace=True,axis=1)
lu.head()

,month,CD,numbldgs,unitstotal,asland_sqft,pct_clean_strts,pct_lot_area_open_space,pct_lot_area_public_facility_institution,pct_lot_area_res_1_2_family_bldg,pct_lot_area_res_multifamily_walkup,restaurant,business,Year,Month,Year_Month
0,2016-11-01,MN01,1760,55988,79.282161,100.0,4.35,31.68,0.17,0.63,3295,683,2016,11,2016-11
1,2016-11-01,MN02,5375,72165,100.324753,98.7,3.55,9.63,4.47,10.26,1271,1013,2016,11,2016-11
2,2016-11-01,MN03,4708,94059,58.236802,93.4,14.95,11.15,0.32,8.66,580,947,2016,11,2016-11
3,2016-11-01,MN04,3923,100090,105.402387,100.0,2.81,11.75,0.88,7.68,964,1014,2016,11,2016-11
4,2016-11-01,MN05,3190,95425,516.066825,100.0,1.59,6.49,0.09,0.67,270,2083,2016,11,2016-11


### 311

In [444]:
df311 = pd.read_csv('311_cleaned.csv', index_col=0)
df311.head()

,CD,Year_Month,Year,Month,Count
0,BK01,2016-11,2016,11,50
1,BK01,2016-12,2016,12,36
2,BK01,2017-01,2017,1,51
3,BK01,2017-02,2017,2,61
4,BK01,2017-03,2017,3,67


In [445]:
df311.rename({'Count':'Count311'},axis=1,inplace=True)
df311.head()

,CD,Year_Month,Year,Month,Count311
0,BK01,2016-11,2016,11,50
1,BK01,2016-12,2016,12,36
2,BK01,2017-01,2017,1,51
3,BK01,2017-02,2017,2,61
4,BK01,2017-03,2017,3,67


In [446]:
df311.shape

(3523, 5)

In [447]:
df311['Year_Month'].nunique()
df311.shape # df311 has different length than the other datasets 

(3523, 5)

In [448]:
# which CDs have incomplete data? 
size = pd.DataFrame(df311.groupby(by='CD').size())
size[size[0] < 60]

,0
CD,
BK02,59
BK07,59
BX03,59
BX04,59
BX05,59
BX08,59
BX11,59
MN05,59
MN06,59


In [449]:
# which months have incomplete data? 
size = pd.DataFrame(df311.groupby(by='Year_Month').size())
size[size[0] < 59]

,0
Year_Month,
2019-07,42


In [450]:
df311 = df311[df311['Year_Month'] != '2021-10']
df311.shape

# 42 rows lost by removing 2021-10.

# not certain if there are exogenic shocks that affected CDs differentially during this month
# such that some CDs did not have make any 311 reports at all, despite
# consistently having done so for all previous months. 
# so a decision was made to remove this month entirely. 

(3464, 5)

## MERGING: The Merge. 

In [451]:
merge1 = swm.merge(df311, on=['Year', 'Month', 'CD'])
merge1.shape

(3464, 28)

In [452]:
merge1.head()

,CD,Year,Month,REFUSETONSCOLLECTED,PAPERTONSCOLLECTED,MGPTONSCOLLECTED,RESORGANICSTONS,fs_openhours,fs_open,trs_permits,...,wfr_avg_bulk,bigbelly,seasonal_wire_litter_basket,highend_basket,publicspacereceptacle,wire_basket,total_litter_baskets,Year_Month_x,Year_Month_y,Count311
0,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,3.0,1.0,0.0,2.0,1.0,10.0,14.0,2016-11,2016-11,50
1,BK01,2016,12,5409.5,726.3,501.0,11.7,333.25,6.0,14.0,...,3.0,1.0,0.0,2.0,1.0,10.0,14.0,2016-12,2016-12,36
2,BK01,2017,1,5189.9,589.1,455.2,14.9,333.25,6.0,14.0,...,3.0,1.0,0.0,2.0,1.0,10.0,14.0,2017-01,2017-01,51
3,BK01,2017,2,4705.4,505.6,395.8,10.5,333.25,6.0,14.0,...,3.0,1.0,0.0,2.0,1.0,10.0,14.0,2017-02,2017-02,61
4,BK01,2017,3,5476.7,563.0,453.3,10.6,333.25,6.0,14.0,...,3.0,1.0,0.0,2.0,1.0,10.0,14.0,2017-03,2017-03,67


In [453]:
merge2 = pd.merge(merge1, ses, on=['Year', 'Month', 'CD'])
merge2.shape

(3464, 40)

In [454]:
merge3 = merge2.merge(lu, on=['Year', 'Month', 'CD'], indicator=True, how='left')
merge3.shape

(3464, 53)

In [455]:
len(merge3[merge3['_merge']!='both']) # check that the merge was all good

0

In [456]:
merge3.columns

Index(['CD', 'Year', 'Month', 'REFUSETONSCOLLECTED', 'PAPERTONSCOLLECTED',
       'MGPTONSCOLLECTED', 'RESORGANICSTONS', 'fs_openhours', 'fs_open',
       'trs_permits', 'paper_baskets', 'mgp_baskets',
       'total_number_of_households', 'hh_compost_num', 'hh_compost_percent',
       'shape_area', 'wfr_avg_recycling', 'wfr_avg_refuse', 'wfr_avg_bulk',
       'bigbelly', 'seasonal_wire_litter_basket', 'highend_basket',
       'publicspacereceptacle', 'wire_basket', 'total_litter_baskets',
       'Year_Month_x', 'Year_Month_y', 'Count311', 'pct_bach_deg',
       'over65_rate', 'under18_rate', 'pct_white_nh', 'poverty_rate',
       'pct_hh_rent_burd', 'mean_commute', 'unemployment', 'crime_per_1000',
       'pop_density', 'month_x', 'Year_Month_x', 'month_y', 'numbldgs',
       'unitstotal', 'asland_sqft', 'pct_clean_strts',
       'pct_lot_area_open_space', 'pct_lot_area_public_facility_institution',
       'pct_lot_area_res_1_2_family_bldg',
       'pct_lot_area_res_multifamily_walkup'

In [472]:
merge3[['Year', 'Month', 'Year_Month_x', 'Year_Month_y']]

,Year,Month,Year_Month_x,Year_Month_x,Year_Month_y,Year_Month_y
0,2016,11,2016-11,2016-11,2016-11,2016-11
1,2016,12,2016-12,2016-12,2016-12,2016-12
2,2017,1,2017-01,2017-01,2017-01,2017-01
3,2017,2,2017-02,2017-02,2017-02,2017-02
4,2017,3,2017-03,2017-03,2017-03,2017-03
...,...,...,...,...,...,...
3459,2021,5,2021-05,2021-05,2021-05,2021-05
3460,2021,6,2021-06,2021-06,2021-06,2021-06
3461,2021,7,2021-07,2021-07,2021-07,2021-07
3462,2021,8,2021-08,2021-08,2021-08,2021-08


In [476]:
merge_final = merge3.drop(['Year_Month_x', 'Year_Month_y',
                           'month_x', 'month_y',
                           '_merge'],axis =1)

In [474]:
merge_final.columns

Index(['CD', 'Year', 'Month', 'REFUSETONSCOLLECTED', 'PAPERTONSCOLLECTED',
       'MGPTONSCOLLECTED', 'RESORGANICSTONS', 'fs_openhours', 'fs_open',
       'trs_permits', 'paper_baskets', 'mgp_baskets',
       'total_number_of_households', 'hh_compost_num', 'hh_compost_percent',
       'shape_area', 'wfr_avg_recycling', 'wfr_avg_refuse', 'wfr_avg_bulk',
       'bigbelly', 'seasonal_wire_litter_basket', 'highend_basket',
       'publicspacereceptacle', 'wire_basket', 'total_litter_baskets',
       'Year_Month_y', 'Count311', 'pct_bach_deg', 'over65_rate',
       'under18_rate', 'pct_white_nh', 'poverty_rate', 'pct_hh_rent_burd',
       'mean_commute', 'unemployment', 'crime_per_1000', 'pop_density',
       'numbldgs', 'unitstotal', 'asland_sqft', 'pct_clean_strts',
       'pct_lot_area_open_space', 'pct_lot_area_public_facility_institution',
       'pct_lot_area_res_1_2_family_bldg',
       'pct_lot_area_res_multifamily_walkup', 'restaurant', 'business',
       'Year_Month_y'],
      dty

In [477]:
# first_column = merge_final.pop('Year_Month_y')
# merge_final.insert(0, 'Year_Month', first_column)

In [478]:
merge_final.head()

,CD,Year,Month,REFUSETONSCOLLECTED,PAPERTONSCOLLECTED,MGPTONSCOLLECTED,RESORGANICSTONS,fs_openhours,fs_open,trs_permits,...,numbldgs,unitstotal,asland_sqft,pct_clean_strts,pct_lot_area_open_space,pct_lot_area_public_facility_institution,pct_lot_area_res_1_2_family_bldg,pct_lot_area_res_multifamily_walkup,restaurant,business
0,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,17147,100971,10.78427,86.4,5.19,5.33,5.25,16.9,217,984
1,BK01,2016,12,5409.5,726.3,501.0,11.7,333.25,6.0,14.0,...,17147,100971,10.78427,86.4,5.19,5.33,5.25,16.9,217,915
2,BK01,2017,1,5189.9,589.1,455.2,14.9,333.25,6.0,14.0,...,17147,100971,10.78427,86.4,5.19,5.33,5.25,16.9,217,921
3,BK01,2017,2,4705.4,505.6,395.8,10.5,333.25,6.0,14.0,...,17147,100971,10.78427,86.4,5.19,5.33,5.25,16.9,217,877
4,BK01,2017,3,5476.7,563.0,453.3,10.6,333.25,6.0,14.0,...,17147,100971,10.78427,86.4,5.19,5.33,5.25,16.9,217,886


In [ ]:
merge_final.shape

(3481, 45)

In [479]:
merge_final.to_csv('final_data.csv')

In [484]:
merge_final.isna().sum().sum()

0